In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sec_edgar.data.preprocessing.preprocess_4form_files import pre_process_4form_archive_files
from sec_edgar.data.preprocessing.preprocess_master_idx import pre_process_master_idx_content
from sec_edgar.data.processing.process_4form_files import Process4FormFiles
from sec_edgar.data.processing.process_append_historical_prices import ProcessAppendHistoricalPrices
from sec_edgar.data.utils.cik_mapping_util import CikMappingUtil
from sec_edgar.stock_historical_data.av_historical_data_request import AVHistoricalDataRequest
import pandas as pd

from sec_edgar.utils.evaluation_metrics import PerformanceMetric01, PerformanceMetric02, PerformanceMetric03
from sec_edgar.utils.performance_evaluations import PerformanceEvaluations4Form, PerformanceEvaluations4FormYear

In [2]:
pd.set_option('display.width', 2500)      # max table width to display

In [3]:
## Loading of data set

In [4]:
data_set_name = "us_market_allderv_AandB_1019.py"
base_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))

path_processed_datasets = base_path + '/Data/processed_datasets/'
file_name = path_processed_datasets + data_set_name + ".csv"

processed_4form_df = pd.read_csv(file_name, index_col=0)

In [5]:
print(file_name)
display(processed_4form_df)
display(processed_4form_df.columns.to_list())

C:\Users\andre\OneDrive\Documents\MathModels\sec_edgar/Data/processed_datasets/us_market_allderv_AandB_1019.py.csv


,CIK,Company_Names,Date_Filed,index_id,index_id_min,n_transactions,shares_percent_changes_mean,transactionSharesAdjust,date_ft_delta_mean,Filename,...,Shifted Price pct_change (10)(10 days 00:00:00),Shifted Price pct_change (10)(-10 days +00:00:00),Shifted Price pct_change (15)(15 days 00:00:00),Shifted Price pct_change (15)(-15 days +00:00:00),Shifted Price pct_change (21)(21 days 00:00:00),Shifted Price pct_change (21)(-21 days +00:00:00),Shifted Price pct_change (31)(31 days 00:00:00),Shifted Price pct_change (31)(-31 days +00:00:00),Shifted Price pct_change (42)(42 days 00:00:00),Shifted Price pct_change (42)(-42 days +00:00:00)
index,,,,,,,,,,,,,,,,,,,,,
48441,27904,DELTA AIR LINES INC /DE/,2010-01-04,[140546],140546,1,NaN,0.00,0 days 00:00:00,['edgar/data/27904/0001019687-10-000008.txt'],...,-0.071374,0.086661,-0.049117,0.075472,-0.043511,0.187556,0.140489,0.352113,0.174694,0.701164
107523,935703,DOLLAR TREE INC,2010-01-04,[578414 578415 578416 578417 578418],578414,5,0.250942,3544.61,3 days 00:00:00,['edgar/data/935703/0001127602-10-000234.txt'\...,...,0.005482,0.000612,-0.034031,0.014925,0.143033,0.017983,0.211652,0.053858,0.232375,-0.000204
48728,28412,COMERICA INC /NEW/,2010-01-04,[79905 79906 79907 79908 79909 79910 79911 799...,79905,13,0.018748,1152.00,5 days 00:00:00,['edgar/data/28412/0000028412-10-000001.txt'\n...,...,0.011134,0.068302,0.030569,0.152212,0.014648,0.167237,0.143071,0.266620,0.203143,0.213538
75418,72741,NORTHEAST UTILITIES,2010-01-04,[378340 378341 378342 378343],378340,4,0.146730,9435.00,0 days 00:00:00,['edgar/data/72741/0000072741-10-000004.txt'\n...,...,-0.040546,-0.000762,-0.051874,0.003069,0.028191,0.022868,0.074148,0.055285,0.109829,0.049897
97444,882835,ROPER INDUSTRIES INC,2010-01-04,[229120 229121 229122 229123],229120,4,-0.027286,0.00,5 days 00:00:00,['edgar/data/882835/0001209191-10-000310.txt'],...,-0.046095,-0.010336,-0.029718,-0.028972,0.074132,-0.028318,0.083082,-0.027359,0.165974,-0.048330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61641,49071,HUMANA INC,2019-12-31,[795842 795844 795845 795846 795847 795848 795...,795842,14,NaN,0.00,0 days 00:00:00,['edgar/data/49071/0000049071-19-000146.txt'\n...,...,-0.047265,-0.005206,-0.013514,0.024926,0.016773,0.056768,-0.189329,0.066934,-0.024946,0.088031
3960,1018724,AMAZON COM INC,2019-12-31,[334103],334103,1,NaN,0.00,4 days 00:00:00,['edgar/data/1018724/0000899243-19-030277.txt'],...,-0.029111,0.067372,0.101078,0.069387,0.138002,0.082767,-0.100004,0.033996,0.008402,0.076832
65091,54480,KANSAS CITY SOUTHERN,2019-12-31,[827929 827930 827932 827933 827934 827935 827...,827929,10,-0.001570,-285.00,11 days 00:00:00,['edgar/data/54480/0000054480-19-000247.txt'],...,0.036840,0.001612,0.076889,0.037715,0.063520,0.046904,-0.281522,0.047828,-0.197721,0.069084


['CIK',
 'Company_Names',
 'Date_Filed',
 'index_id',
 'index_id_min',
 'n_transactions',
 'shares_percent_changes_mean',
 'transactionSharesAdjust',
 'date_ft_delta_mean',
 'Filename',
 'Form_Type',
 'conversionOrExercisePrice',
 'date_ft_delta',
 'deemedExecutionDate',
 'directOrIndirectOwnership',
 'doc_name',
 'exerciseDate',
 'expirationDate',
 'file_url',
 'isDirector',
 'isOfficer',
 'isOther',
 'isTenPercentOwner',
 'issuerName',
 'issuerTradingSymbol',
 'officerTitle',
 'otherText',
 'periodOfReport',
 'quarter',
 'rptOwnerName',
 'securityTitle',
 'sharesOwnedFollowingTransaction',
 'shares_percent_changes',
 'signatureDate',
 'transactionAcquiredDisposedCode',
 'transactionCode',
 'transactionDate',
 'transactionFormType',
 'transactionPricePerShare',
 'transactionShares',
 'transactionTotalValue',
 'transaction_table',
 'transaction_type',
 'underlyingSecurityShares',
 'underlyingSecurityTitle',
 'underlyingSecurityValue',
 'valueOwnedFollowingTransaction',
 'year',
 'my_de

## Performance Metrics Evaluations

In [6]:
pe_4form = PerformanceEvaluations4Form(processed_4form_df, n_sub_set=5)

In [7]:
days = 10
column_label_eval = 'Price pct_change ({days})'.format(days=days)
shifted_columns_label_eval = 'Shifted Price pct_change ({days})(-{days} days +00:00:00)'.format(days=days)

In [8]:

# PerformanceMetric01 evaluation:
metrics_df_01, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric01,
                                        pm_kwargs=dict(column_label_eval=column_label_eval))
print("\nPerformanceMetric01 evaluation: ")
display(metrics_df_01)
print("-" * 75)

# PerformanceMetric02 evaluation:
metrics_df_02, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric02,
                                        pm_kwargs=dict(column_label_eval=column_label_eval,
                                                       shifted_columns_label_eval=
                                                       shifted_columns_label_eval))

metrics_df_02b, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric02,
                                         pm_kwargs=dict(column_label_eval=column_label_eval,
                                                        shifted_columns_label_eval=
                                                        'Shifted Price pct_change (10)(10 days 00:00:00)'))
print("\nPerformanceMetric02 evaluation: ")
display(metrics_df_02)
print("-" * 75)

# PerformanceMetric03 evaluation:
metrics_df_03, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric03,
                                        pm_kwargs=dict(column_label_eval=column_label_eval,
                                                       shifted_columns_label_eval=
                                                       shifted_columns_label_eval))

metrics_df_03b, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric03,
                                         pm_kwargs=dict(column_label_eval=column_label_eval,
                                                        shifted_columns_label_eval=
                                                        'Shifted Price pct_change (10)(10 days 00:00:00)'))

print("\nPerformanceMetric03 evaluation: ")
display(metrics_df_03)
print("-" * 75)


PerformanceMetric01 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.005882,0.00786,0.003699,0.006231,0.005068,0.006551
benchmark,0.004483,0.005515,0.004239,0.004571,0.003934,0.004155
is_greater_prob,0.621176,0.705882,0.517647,0.670588,0.611765,0.6
is_pos_tse_ok,True,True,False,True,True,True
is_greater_prob_ok,True,True,True,True,True,True
is_performance_ok,True,True,False,True,True,True
Positive_tsa_std,0.010501,0.007766,0.009778,0.008553,0.009529,0.015058
benchmark_std,0.005508,0.004652,0.005432,0.004828,0.00391,0.007864
statistic,2.432432,2.38825,-0.444653,1.558457,1.014981,1.300382
pvalue,0.007635,0.009144,0.671349,0.060754,0.156156,0.097916


---------------------------------------------------------------------------

PerformanceMetric02 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.005882,0.00786,0.003699,0.006231,0.005068,0.006551
benchmark,0.004443,0.006546,0.002891,0.005847,0.003432,0.003498
is_greater_prob,0.571765,0.670588,0.552941,0.552941,0.517647,0.564706
is_pos_tse_ok,True,True,True,True,True,True
is_greater_prob_ok,True,True,True,True,True,True
is_performance_ok,True,True,True,True,True,True
Positive_tsa_std,0.010501,0.007766,0.009778,0.008553,0.009529,0.015058
benchmark_std,0.014932,0.019257,0.011177,0.011727,0.011942,0.018403
statistic,1.625264,0.583601,0.501715,0.24392,0.987087,1.183928
pvalue,0.05226,0.280339,0.308268,0.403809,0.162545,0.11909


---------------------------------------------------------------------------

PerformanceMetric03 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.006018,0.007499,0.004891,0.006314,0.005157,0.00581
benchmark,0.004603,0.00649,0.003314,0.005045,0.003873,0.003559
is_greater_prob,0.500934,0.509697,0.496723,0.494208,0.495981,0.508117
is_pos_tse_ok,True,True,True,True,True,True
is_greater_prob_ok,True,True,False,False,False,True
is_performance_ok,True,True,False,False,False,True
Positive_tsa_std,0.057361,0.06677,0.053566,0.056209,0.051763,0.053088
benchmark_std,0.085695,0.136061,0.056549,0.061638,0.058088,0.064704
statistic,2.031948,0.494523,1.436748,1.02844,1.008364,1.492507
pvalue,0.021083,0.310475,0.07541,0.151885,0.156656,0.06781


---------------------------------------------------------------------------


In [9]:
print("=" * 90)
pe_4form_year = PerformanceEvaluations4FormYear(processed_4form_df)

# PerformanceMetric01 evaluation:
metrics_df_01_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric01,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval))
print("\nPerformanceMetric01 evaluation (Year): ")
display(metrics_df_01_year)
print("-" * 75)

# PerformanceMetric02 evaluation:
metrics_df_02_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric02,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                 shifted_columns_label_eval=
                                                                 shifted_columns_label_eval))

metrics_df_02b_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric02,
                                                   pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                  shifted_columns_label_eval=
                                                                  'Shifted Price pct_change (10)(10 days 00:00:00)'))
print("\nPerformanceMetric02 evaluation (Year): ")
display(metrics_df_02_year)
print("-" * 75)

# PerformanceMetric03 evaluation:
metrics_df_03_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric03,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                 shifted_columns_label_eval=
                                                                 shifted_columns_label_eval))

metrics_df_03b_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric03,
                                                   pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                  shifted_columns_label_eval=
                                                                  'Shifted Price pct_change (10)(10 days 00:00:00)'))

print("\nPerformanceMetric03 evaluation (Year): ")
display(metrics_df_03_year)
print("-" * 75)

print("=" * 90)


PerformanceMetric01 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.005882,0.007131,-0.001945,0.007714,0.013262,0.004211,0.000416,0.012244,0.006811,0.002351,0.008629
benchmark,0.004483,0.006174,-0.004343,0.006455,0.008191,0.005241,-0.002719,0.009867,0.005614,-0.00197,0.006441
is_greater_prob,0.621176,0.545685,0.553885,0.534005,0.61596,0.523691,0.586797,0.524272,0.528846,0.57554,0.563246
is_pos_tse_ok,True,True,True,True,True,False,True,True,True,True,True
is_greater_prob_ok,True,True,True,True,True,True,True,True,True,True,True
is_performance_ok,True,True,True,True,True,False,True,True,True,True,True
Positive_tsa_std,0.010501,0.034339,0.035548,0.025832,0.029459,0.051812,0.027715,0.032743,0.023868,0.030582,0.029824
benchmark_std,0.005508,0.018608,0.018973,0.013016,0.014006,0.013061,0.014853,0.019669,0.016871,0.016752,0.016909
statistic,2.432432,0.486423,1.189177,0.866686,3.1132,-0.386031,2.016191,1.263006,0.835486,2.530497,1.306733
pvalue,0.007635,0.313422,0.117417,0.193235,0.000972,0.650172,0.022104,0.103512,0.201856,0.005813,0.095879


---------------------------------------------------------------------------

PerformanceMetric02 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.005882,0.007131,-0.001945,0.007714,0.013262,0.004211,0.000416,0.012244,0.006811,0.002351,0.008629
benchmark,0.004443,0.008411,0.004366,0.00729,0.011891,0.006437,-0.002174,0.007987,0.008284,-0.005633,0.010911
is_greater_prob,0.571765,0.535533,0.506266,0.523929,0.523691,0.513716,0.535452,0.531553,0.46875,0.573141,0.46778
is_pos_tse_ok,True,False,False,True,True,False,True,True,False,True,False
is_greater_prob_ok,True,True,True,True,True,True,True,True,False,True,False
is_performance_ok,True,False,False,True,True,False,True,True,False,True,False
Positive_tsa_std,0.010501,0.034339,0.035548,0.025832,0.029459,0.051812,0.027715,0.032743,0.023868,0.030582,0.029824
benchmark_std,0.014932,0.034909,0.08601,0.031179,0.029287,0.02935,0.031502,0.032721,0.027043,0.036826,0.03274
statistic,1.625264,-0.518876,-1.354519,0.208216,0.660955,-0.748369,1.248189,1.866335,-0.832865,3.406096,-1.054594
pvalue,0.05226,0.698003,0.911926,0.417558,0.254416,0.772742,0.106163,0.031175,0.797418,0.000346,0.854041


---------------------------------------------------------------------------

PerformanceMetric03 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.006018,0.008849,-0.001228,0.006245,0.012076,0.005951,0.000973,0.011419,0.005746,0.002106,0.008634
benchmark,0.004603,0.008104,0.002997,0.005118,0.010143,0.007027,-0.004141,0.0062,0.006647,-0.005249,0.009563
is_greater_prob,0.500934,0.507836,0.493501,0.50024,0.499032,0.475826,0.518553,0.513361,0.483047,0.528343,0.487858
is_pos_tse_ok,True,True,False,True,True,False,True,True,False,True,False
is_greater_prob_ok,True,True,False,True,False,False,True,True,False,True,False
is_performance_ok,True,True,False,True,False,False,True,True,False,True,False
Positive_tsa_std,0.057361,0.061734,0.091353,0.051421,0.045146,0.048547,0.049464,0.059795,0.038778,0.05351,0.05202
benchmark_std,0.085695,0.060634,0.205934,0.052834,0.04713,0.04706,0.05946,0.060212,0.051873,0.062907,0.06184
statistic,2.031948,0.417923,-0.90101,0.697961,1.346494,-0.727522,3.05048,2.840563,-0.645463,4.281213,-0.551515
pvalue,0.021083,0.338011,0.816174,0.24262,0.089109,0.766527,0.00115,0.002262,0.740668,0.000009,0.709346


---------------------------------------------------------------------------


## OutLayers Rem Evaluation test

In [10]:
post_processed_4form_df = processed_4form_df.copy()
post_processed_4form_df = post_processed_4form_df[post_processed_4form_df.my_derivative_types.isin(['AB', 'B'])]
post_processed_4form_df = post_processed_4form_df[post_processed_4form_df['transactionSharesAdjust'] > 0]

In [11]:
def _ecdf(sample):
    sample = np.atleast_1d(sample)
    quantiles, counts = np.unique(sample, return_counts=True)
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob


def _get_sub_set_in_ci(quantiles, cumprob, ci=10 / 100):
    return quantiles[np.logical_and(cumprob > 0.5 * ci, cumprob < 1. - 0.5 * ci)]

def _get_left_sub_set_in_ci(quantiles, cumprob, ci=10 / 100):
    return quantiles[cumprob < 1. - ci]

quantiles, cumprob = _ecdf(post_processed_4form_df[column_label_eval].values)
results_dict = dict()
results_dict["E[R|0.05<P(R)<0.95]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.05 * 2).mean()
results_dict["E[R|0.10<P(R)<0.90]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.1 * 2).mean()
results_dict["E[R|0.15<P(R)<0.85]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.15 * 2).mean()
results_dict["E[R|0.20<P(R)<0.80]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.2 * 2).mean()

quantiles, cumprob = _ecdf(post_processed_4form_df[shifted_columns_label_eval].values)
results_dict_bench = dict()
results_dict_bench["E[R|0.05<P(R)<0.95]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.05 * 2).mean()
results_dict_bench["E[R|0.10<P(R)<0.90]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.1 * 2).mean()
results_dict_bench["E[R|0.15<P(R)<0.85]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.15 * 2).mean()
results_dict_bench["E[R|0.20<P(R)<0.80]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.2 * 2).mean()

In [12]:
print("\n MostProb Scenarios results: \n")
display(pd.Series(results_dict))
print("\n MostProb Scenarios results Bench: \n")
display(pd.Series(results_dict_bench))


 MostProb Scenarios results: 



E[R|0.05<P(R)<0.95]    0.006596
E[R|0.10<P(R)<0.90]    0.006735
E[R|0.15<P(R)<0.85]    0.006851
E[R|0.20<P(R)<0.80]    0.006917
E[R|0.25<P(R)<0.75]    0.006965
dtype: float64


 MostProb Scenarios results Bench: 



E[R|0.05<P(R)<0.95]    0.004625
E[R|0.10<P(R)<0.90]    0.005014
E[R|0.15<P(R)<0.85]    0.005296
E[R|0.20<P(R)<0.80]    0.005508
E[R|0.25<P(R)<0.75]    0.005658
dtype: float64